Reference -- https://colab.research.google.com/drive/1j7Kv1qKsXmIhWXlvZ4Jm_6K9DJ1pBgHp?usp=sharing

In [1]:
# !pip install -q langchain==0.0.208 openai==0.27.8 python-dotenv

In [2]:
# Load environment variables
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

True

## PydanticOutputParser

In [3]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser

In [4]:
from pydantic import BaseModel, Field, validator
from typing import List

In [5]:
model_name = "text-davinci-003"
temperature = 0
model = OpenAI(model_name=model_name, temperature=temperature)

## Documentation Example

In [7]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @field_validator('setup')
    def question_ends_with_question_mark(cls, field):
        if field[-1] != '?':
            raise ValueError("Badly formed question!")
        return field

NameError: name 'field_validator' is not defined

In [8]:
# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

In [9]:
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

NameError: name 'Joke' is not defined

In [10]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

_input = prompt.format_prompt(query=joke_query)

NameError: name 'parser' is not defined

In [11]:
output = model(_input.to_string())

NameError: name '_input' is not defined

In [12]:
parser.parse(output)

NameError: name 'parser' is not defined

In [13]:
# Here's another example, but with a compound typed field.
class Actor(BaseModel):
    name: str = Field(description="name of an actor")
    film_names: List[str] = Field(description="list of names of films they starred in")

In [14]:
actor_query = "Generate the filmography for a random actor."

In [15]:
parser = PydanticOutputParser(pydantic_object=Actor)

In [16]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [17]:
_input = prompt.format_prompt(query=actor_query)

In [18]:
output = model(_input.to_string())

NotFoundError: Error code: 404 - {'error': {'message': 'The model `text-davinci-003` has been deprecated, learn more here: https://platform.openai.com/docs/deprecations', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

In [19]:
parser.parse(output)

NameError: name 'output' is not defined